In [1]:
!pip install bcrypt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 5.9 MB/s eta 0:00:00


# 1. Database Initialization

In [2]:
# 📦 Initialize Database with Users, Transactions, and Loans tables
import sqlite3
import bcrypt
from datetime import datetime

def init_db():
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    # Create Users table
    c.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT,
        role TEXT,
        balance REAL DEFAULT 0.0
    )''')
    # Create Transactions table
    c.execute('''CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        type TEXT,
        amount REAL,
        date TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    )''')
    # Create Loans table
    c.execute('''CREATE TABLE IF NOT EXISTS loans (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        amount REAL,
        status TEXT,
        date_applied TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    )''')
    conn.commit()
    conn.close()
    c.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT,
        role TEXT,
        balance REAL DEFAULT 0.0
    )''')
    # Create Transactions table
    c.execute('''CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        type TEXT,
        amount REAL,
        date TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    )''')
    # Create Loans table
    c.execute('''CREATE TABLE IF NOT EXISTS loans (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        amount REAL,
        status TEXT,
        date_applied TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    )''')
    conn.commit()
    conn.close()


# 2. Authentication: Signup and Login

In [3]:
# 📝 Signup a new user (Customer or Admin)
def signup(username, password, role_input):
    role_input = role_input.strip().lower()
    if role_input in ['c', 'customer']:
        role = 'customer'
    elif role_input in ['a', 'admin']:
        role = 'admin'
    else:
        print("Invalid input. Please enter 'C' for Customer or 'A' for Admin.")
        return

    hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)", (username, hashed, role))
        conn.commit()
        print(f"Signup successful as {role.capitalize()}!")
    except sqlite3.IntegrityError:
        print("Username already exists!")
    conn.close()

# 🔑 Login user and validate credentials and role
def login(username, password, role_input):
    role_input = role_input.strip().lower()
    if role_input in ['c', 'customer']:
        role = 'customer'
    elif role_input in ['a', 'admin']:
        role = 'admin'
    else:
        print("Invalid input. Please enter 'C' for Customer or 'A' for Admin.")
        return None

    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT id, password FROM users WHERE username=? AND role=?", (username, role))
    user = c.fetchone()
    conn.close()
    if user and bcrypt.checkpw(password.encode(), user[1]):
        print(f"Welcome {username}! Logged in as {role.capitalize()}.")
        return (user[0], username, role)
    else:
        print("Invalid credentials or role.")
        return None


# 3. Admin Functionalities

In [4]:
# 👤 Create a new customer account
def create_customer(username, password):
    signup(username, password, 'c')

# ❌ Delete a customer account
def delete_customer(username):
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE username=? AND role='customer'", (username,))
    conn.commit()
    conn.close()
    print("Customer account deleted.")

# ✏️ Update customer username
def update_customer(username, new_contact=None):
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    if new_contact:
        c.execute("UPDATE users SET username=? WHERE username=? AND role='customer'", (new_contact, username))
        conn.commit()
        print("Customer information updated.")
    else:
        print("No updates made.")
    conn.close()

# 📋 List all customer users
def list_customers():
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT id, username, balance FROM users WHERE role='customer'")
    for row in c.fetchall():
        print(row)
    conn.close()

# 🔎 Search customers by balance range
def search_customers_by_balance(min_bal, max_bal):
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT username, balance FROM users WHERE balance BETWEEN ? AND ?", (min_bal, max_bal))
    for row in c.fetchall():
        print(row)
    conn.close()

# 📑 View transactions of a specific customer
def view_customer_transactions(username):
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT id FROM users WHERE username=?", (username,))
    uid = c.fetchone()
    if uid:
        c.execute("SELECT type, amount, date FROM transactions WHERE user_id=?", (uid[0],))
        for row in c.fetchall():
            print(f"{row[0]} - ₹{row[1]} on {row[2]}")
    else:
        print("Customer not found.")
    conn.close()

# 🏦 Manage loan approvals/rejections
def manage_loans():
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT loans.id, users.username, loans.amount, loans.status FROM loans JOIN users ON loans.user_id = users.id")
    loans = c.fetchall()
    for loan in loans:
        print(f"Loan ID: {loan[0]}, User: {loan[1]}, Amount: ₹{loan[2]}, Status: {loan[3]}")
    loan_id = input("Enter Loan ID to approve/reject (or press Enter to skip): ")
    if loan_id:
        action = input("Approve (A) or Reject (R): ").strip().lower()
        status = 'approved' if action == 'a' else 'rejected'
        c.execute("UPDATE loans SET status=? WHERE id=?", (status, loan_id))
        conn.commit()
        print(f"Loan {loan_id} has been {status}.")
    conn.close()

# 📊 Generate report of customer and loan statistics
def generate_report():
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM users WHERE role='customer'")
    users_count = c.fetchone()[0]
    c.execute("SELECT SUM(balance) FROM users WHERE role='customer'")
    total_balance = c.fetchone()[0]
    c.execute("SELECT status, COUNT(*) FROM loans GROUP BY status")
    loan_stats = c.fetchall()
    print(f"Total Customers: {users_count}")
    print(f"Total Balance Held: ₹{total_balance}")
    for stat in loan_stats:
        print(f"Loans {stat[0].capitalize()}: {stat[1]}")
    conn.close()


# 4. Customer Functionalities

In [5]:
# 💰 Deposit money into user's account
def deposit(user, amount):
    uid, _, _ = user
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("UPDATE users SET balance = balance + ? WHERE id=?", (amount, uid))
    c.execute("INSERT INTO transactions (user_id, type, amount, date) VALUES (?, 'deposit', ?, ?)",
              (uid, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()
    conn.close()
    print("Deposit successful.")

# 💸 Withdraw money and show updated balance
def withdraw(user, amount):
    uid, _, _ = user
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT balance FROM users WHERE id=?", (uid,))
    balance = c.fetchone()[0]
    if balance >= amount:
        c.execute("UPDATE users SET balance = balance - ? WHERE id=?", (amount, uid))
        c.execute("INSERT INTO transactions (user_id, type, amount, date) VALUES (?, 'withdraw', ?, ?)",
                  (uid, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit()
        c.execute("SELECT balance FROM users WHERE id=?", (uid,))
        updated_balance = c.fetchone()[0]
        print(f"Withdrawal successful. New Balance: ₹{updated_balance}")
    else:
        print("Insufficient balance.")
    conn.close()

# 🔁 Transfer funds between users
def transfer_money(sender, receiver_username, amount):
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT id, balance FROM users WHERE username=?", (sender[1],))
    sender_data = c.fetchone()
    if sender_data:
        sid, sender_balance = sender_data
    else:
        print("Sender not found.")
        conn.close()
        return

    c.execute("SELECT id FROM users WHERE username=?", (receiver_username,))
    receiver = c.fetchone()
    if receiver:
        rid = receiver[0]
        if sender_balance >= amount:
            c.execute("UPDATE users SET balance = balance - ? WHERE id=?", (amount, sid))
            c.execute("UPDATE users SET balance = balance + ? WHERE id=?", (amount, rid))
            c.execute("INSERT INTO transactions (user_id, type, amount, date) VALUES (?, 'transfer', ?, ?)",
                      (sid, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
            conn.commit()
            print("Transfer successful.")
        else:
            print("Insufficient funds.")
    else:
        print("Receiver not found.")
    conn.close()

# 📃 View logged-in user's transaction history
def view_transactions(user):
    uid, _, _ = user
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("SELECT type, amount, date FROM transactions WHERE user_id=?", (uid,))
    records = c.fetchall()
    if records:
        for row in records:
            print(f"{row[0].capitalize()} - ₹{row[1]} on {row[2]}")
    else:
        print("No transactions found.")
    conn.close()

# 🧾 Submit a loan application
def apply_loan(user, amount):
    uid, _, _ = user
    conn = sqlite3.connect('banking.db')
    c = conn.cursor()
    c.execute("INSERT INTO loans (user_id, amount, status, date_applied) VALUES (?, ?, 'pending', ?)",
              (uid, amount, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()
    conn.close()
    print("Loan application submitted.")


# Main Application Loop

In [6]:
import sqlite3

def init_db():
    conn = sqlite3.connect('bank.db')
    c = conn.cursor()

    # Create users table
    c.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT,
        role TEXT
    )''')

    # Create accounts table
    c.execute('''CREATE TABLE IF NOT EXISTS accounts (
        user_id INTEGER,
        balance REAL DEFAULT 0,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )''')

    # Create transactions table
    c.execute('''CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sender TEXT,
        receiver TEXT,
        amount REAL,
        timestamp TEXT
    )''')

    # Create loans table
    c.execute('''CREATE TABLE IF NOT EXISTS loans (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        amount REAL,
        status TEXT,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )''')

    conn.commit()
    conn.close()


In [ ]:
# 🏁 Main control flow of the banking system
def main():
    init_db()
    print("\U0001F3E6 Welcome to the Banking Management System")
    while True:
        print("\n1. Sign Up\n2. Login\n3. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == '1':
            uname = input("Enter username: ")
            pwd = input("Enter password: ")
            role = input("Enter role (C for Customer, A for Admin): ")
            signup(uname, pwd, role)

        elif choice == '2':
            uname = input("Enter username: ")
            pwd = input("Enter password: ")
            role = input("Enter role (C for Customer, A for Admin): ")
            user = login(uname, pwd, role)
            if user:
                uid, username, role = user
                while True:
                    print(f"\n--- {role.capitalize()} Dashboard ---")
                    if role == 'admin':
                        print("1. Transfer Funds")
                        print("2. View Transactions")
                        print("3. Create Customer")
                        print("4. Delete Customer")
                        print("5. Update Customer")
                        print("6. List Customers")
                        print("7. Search Customers by Balance")
                        print("8. View Customer Transactions")
                        print("9. Manage Loans")
                        print("10. View Reports")
                        print("11. Logout")
                        dash_choice = input("Choose an option: ").strip()
                        if dash_choice == '1':
                            sender = input("Sender Username: ")
                            receiver = input("Receiver Username: ")
                            amount = float(input("Amount to transfer: "))
                            transfer_funds((None, sender, 'admin'), receiver, amount)
                        elif dash_choice == '2':
                            uname = input("Username to view transactions: ")
                            view_customer_transactions(uname)
                        elif dash_choice == '3':
                            uname = input("New customer username: ")
                            pwd = input("Password: ")
                            create_customer(uname, pwd)
                        elif dash_choice == '4':
                            uname = input("Customer username to delete: ")
                            delete_customer(uname)
                        elif dash_choice == '5':
                            uname = input("Existing username: ")
                            newname = input("New username: ")
                            update_customer(uname, newname)
                        elif dash_choice == '6':
                            list_customers()
                        elif dash_choice == '7':
                            minb = float(input("Minimum balance: "))
                            maxb = float(input("Maximum balance: "))
                            search_customers_by_balance(minb, maxb)
                        elif dash_choice == '8':
                            uname = input("Customer username: ")
                            view_customer_transactions(uname)
                        elif dash_choice == '9':
                            manage_loans()
                        elif dash_choice == '10':
                            generate_report()
                        elif dash_choice == '11':
                            print("Logging out...")
                            break
                        else:
                            print("Invalid admin option. Try again.")
                    else:
                        print("1. Deposit")
                        print("2. Withdraw")
                        print("3. Transfer Money")
                        print("4. View Transactions")
                        print("5. Apply for Loan")
                        print("6. Logout")
                        dash_choice = input("Choose an option: ").strip()
                        if dash_choice == '1':
                            amt = float(input("Amount to deposit: "))
                            deposit(user, amt)
                        elif dash_choice == '2':
                            amt = float(input("Amount to withdraw: "))
                            withdraw(user, amt)
                        elif dash_choice == '3':
                            receiver = input("Receiver username: ")
                            amt = float(input("Amount to transfer: "))
                            transfer_funds(user, receiver, amt)
                        elif dash_choice == '4':
                            view_transactions(user)
                        elif dash_choice == '5':
                            amt = float(input("Loan amount: "))
                            apply_loan(user, amt)
                        elif dash_choice == '6':
                            print("Logging out...")
                            break
                        else:
                            print("Invalid option. Try again.")

        elif choice == '3':
            print("Exiting... Goodbye!")
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


🏦 Welcome to the Banking Management System

1. Sign Up
2. Login
3. Exit
